In [16]:
import numpy as np
import scipy.sparse as sp
import scipy.io as spio
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import pearsonr

In [17]:
#Read splicing data

df = pd.read_csv('Splicing_Data.txt', sep='\t')
df = df.loc[~np.isnan(df.SD1_Usage)].copy().reset_index(drop=True)

df = df.iloc[-10000:].copy().reset_index(drop=True)

df['Region1'] = df['Seqs'].str.slice(2, 37)

print('Size of splicing dataset = ' + str(len(df)))

Size of splicing dataset = 10000


In [18]:
#Generate 6mer feature matrix

mer6_dict = {}
mer6_list = []
bases = list('ACGT')

#Build dictionary of 6-mer -> index
i = 0
for b1 in bases :
    for b2 in bases :
        for b3 in bases :
            for b4 in bases :
                for b5 in bases :
                    for b6 in bases :
                        mer6_dict[b1 + b2 + b3 + b4 + b5 + b6] = i
                        mer6_list.append(b1 + b2 + b3 + b4 + b5 + b6)
                        i += 1

#Loop over dataframe, fill matrix X with 6-mer counts
X = sp.lil_matrix((len(df), len(mer6_dict)))
for index, row in df.iterrows() :
    if index % 2000 == 0 :
        print('Extracting 6-mer features from sequence ' + str(index))
    
    region1 = row['Region1']
    #Loop over all 6-mers in the current sequence
    for j in range(0, len(region1) - 6 + 1) :
        if region1[j:j+6] in mer6_dict :
            #Increment X at the corresponding 6-mer index position
            X[index, mer6_dict[region1[j:j+6]]] += 1.

X = sp.csr_matrix(X)
y = np.ravel(df['SD1_Usage'].values)

print('Shape of X = ' + str(X.shape))
print('Shape of y = ' + str(y.shape))


Extracting 6-mer features from sequence 0
Extracting 6-mer features from sequence 2000
Extracting 6-mer features from sequence 4000
Extracting 6-mer features from sequence 6000
Extracting 6-mer features from sequence 8000
Shape of X = (10000, 4096)
Shape of y = (10000,)


In [19]:
#Problem 1.1
#TODO: Calculate log odds ratios of 6-mers using feature matrix X and splicing ratios y

X_col = sp.csc_matrix(X) #More efficient representation of X when working with columns
logodds_ratios = np.zeros(X_col.shape[1])

#Loop over every 6-mer index
for j in range(logodds_ratios.shape[0]) :
    j_nonzero = X_col[:,j].nonzero()[0]
    mask = np.ones(X_col.shape[0])
    mask[j_nonzero] = 0
    j_zero = mask.nonzero()[0]
    if len(j_nonzero) == 0:
        y_has_j = 0
    else:
        y_has_j = np.sum(y[j_nonzero]) / len(j_nonzero)
    y_not_j = np.sum(y[j_zero]) / len(j_zero)
    
    logodds_ratios[j] = np.log((y_has_j/(1 - y_has_j)) / (y_not_j/(1 - y_not_j)))
    
    if j % 1000 == 0 :
        print(f'Calculating logodds for 6-mer {j}, {mer6_list[j]}, {logodds_ratios[j]}')


Calculating logodds for 6-mer 0, AAAAAA, -0.10972614099450448
Calculating logodds for 6-mer 1000, ATTGGA, -0.594009220241114


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


Calculating logodds for 6-mer 2000, CTTCAA, 0.17423057558672736
Calculating logodds for 6-mer 3000, GTGTGA, -0.2820610383732596
Calculating logodds for 6-mer 4000, TTGGAA, -0.4075865029909619


In [ ]:
#Problem 1.1
#TODO: Plot the sorted Log odds ratios, and print the smallest and largest values
odds_sort_index = np.argsort(logodds_ratios, axis=0)
sorted_odds = logodds_ratios[odds_sort_index]
sorted_mers = np.array(mer6_list)[odds_sort_index]
print(f'minimum log odds : {sorted_mers[0]}, {sorted_odds[0]}')
print(f'maximum log odds : {sorted_mers[len(sorted_mers) - 1]}, {sorted_odds[len(sorted_odds) - 1]}')


f = plt.figure(figsize=(12,8))
plt.scatter(sorted_odds, sorted_mers)
plt.title("Log odds ratios sorted", fontsize=16)
plt.xlabel('Log odds ratio', fontsize=14)
plt.ylabel('6mer', fontsize=14)
plt.yticks(np.arange(0, len(sorted_mers), step=256))
plt.tight_layout()
plt.show()

In [ ]:
#Problem 1.3
#TODO: Split data (matrix X and vector y) into training and test sets. Test set should contain 2,000 data points
split = 2000
trainX = X[split:]
testX = X[:split]
trainY = y[split:]
testY = y[:split]


In [ ]:
#Problem 1.3
#TODO: Implement Gradient Descent with KL-divergence gradients for regressing splice site usages

#Helper function for computing log(x / y) in a safe way (whenever x or y is 0).
def safe_kl_log(num, denom) :
    log_vec = np.zeros(num.shape)
    log_vec[(num > 0) & (denom > 0)] = np.log(num[(num > 0) & (denom > 0)] / denom[(num > 0) & (denom > 0)])
    
    return log_vec

#Compute the KL divergence loss (alpha is regularization parameter)
def kl_divergence_loss(X, w, w_0, y_true, alpha=0.0001) :
    #TODO: Implement and return kl divergence loss function L(w, w0, alpha)
    y_guesses = np.zero(y_true)
    for i in range(X.shape[0]):
        tmp = w0 + np.sum(w * X[i])
        y_guesses[i] = 1 / (1 + (e**-tmp))
    num_sum = np.sum(y_true * safe_kl_log(y_true, y_guesses) + (np.ones(y_true.shape) - y_true) * safe_kl_log(np.ones(y_true.shape) - ytrue, np.ones(y_true.shape) - y_guesses))
    num_sum2 = 0
    for i in range(X.shape[0]):
        num_sum2 += y_true[i] * safe_kl_log(y_true[i], y_guesses[i]) + (1 - y_true[i]) * safe_kl_log(1 - y_true[i], 1 - y_guesses[i])
    print(f'{num_sum} == {num_sum2}')
    L = num_sum / len(y_guesses) + alpha * np.sum([wj ** 2 for wj in w]) / 2
    return L

#Compute the KL divergence gradients for the weight vector w and intercept term w_0 (alpha is regularization parameter)
def kl_divergence_gradients(X, w, w_0, y_true, alpha=0.0001) :
    #TODO: Implement and return kl divergence loss gradients for w and w_0
    y_guesses = np.zero(y_true)
    for i in range(X.shape[0]):
        tmp = w0 + np.sum(w * X[i])
        y_guesses[i] = 1 / (1 + (e**-tmp))
    w_grad = np.zeros(w.shape)
    for w_k in range(len(w)):
        w_grad[w_k] = np.sum(X[:,w_k] * (y_guesses - y_true)) / len(y_true) + alpha * w[w_k]
    w0_grad = np.sum(y_guesses - y_true) / len(y_true)
    
    return w_grad, w_0

#Gradient Descent algorithm to optimize weights w and w_0
def gradient_descent(X_train, y_train, X_test, y_test, w, w_0, step_size=0.1, alpha=0.0001, max_epochs=2000) :
    
    mean_train_losses = []
    mean_test_losses = []
    for epoch in range(max_epochs) : #Stop after unreasonable # of iterations, in case we never converge
        if epoch % 50 == 1 and len(mean_train_losses) > 0 :
            print('Training epoch = ' + str(epoch))
            print('Training set KL-div = ' + str(round(mean_train_losses[-1], 4)))
            print('Test set KL-div = ' + str(round(mean_test_losses[-1], 4)))
        
        #TODO: Calculate the KL loss and gradients on the training set
        kl_loss_train = kl_divergence_loss(X_train, w, w_0, y_train)
        w_grad, w0_grad = kl_divergence_gradients(X_train, w, w_0, y_train)
        
        #Update your weights w and w_0 based on the gradients
        w = w - step_size * w_grad
        w_0 = w_0 - step_size * w0_grad
        
        #Append your mean train and test loss to 'mean_train_losses' and 'mean_test_losses'
        kl_loss_test = kl_divergence_loss(X_test, w, w_0, y_test)
        mean_train_losses.append(kl_loss_train)
        mean_test_losses.append(kl_loss_test)
        
        #TODO: Stop the loop once the training loss stops decreasing significantly
        if mean_train_losses[-2] - mean_train_losses[-1] < alpha:
            break

    print('Gradient descent completed.')
    print('Final training set KL-div = ' + str(round(mean_train_losses[-1], 4)))
    print('Final test set KL-div = ' + str(round(mean_test_losses[-1], 4)))
    
    return w, w_0, mean_train_losses, mean_test_losses

In [ ]:
#Problem 1.3
#TODO: Plot training and test set loss (mean KL-div) vs. training iteration
w = np.zeros()
w_0 = 0
w, w_0, mean_train_losses, mean_test_losses = gradient_descent(trainX, trainY, testX, testY, w, w0)


f = plt.figure(figsize=(12,8))
plt.scatter(range(len(mean_test_losses)), mean_test_losses)
plt.title("Test set loss vs. training iteration", fontsize=16)
plt.xlabel('Training iteration', fontsize=14)
plt.ylabel('Test set loss', fontsize=14)
plt.tight_layout()
plt.show()

#TODO: Scatter plot of true vs. pred SD1 usage on test set, and print R^2 coefficient.


In [ ]:
#Problem 1.3
#TODO: Plot the 10 6-mers and corresponding weights of largest magnitude
#TODO: Plot the 10 6-mers and corresponding weights of smallest magnitude
